In [ ]:
from msa2qubo import *
from qubo2msa import *
import numpy as np

In [ ]:
m2q = Msa2Qubo(input="./seq.fa", output="t2/qmsa.qubo", P=1, verbose=False, delta=2.0, l0=1.0, l1=0.1, l2=10.0)

In [ ]:
m2q.run()
print(m2q.bvc)

In [ ]:
import math

In [ ]:
X = m2q.bvc.x()
print("seqs=",X)
print("# seqs=",len(X))
nr = 0
ny = 0
nz = 0
for k in range(0,len(X)-1):
    for q in range(k+1, len(X)):
        for i in range(0, len(X[k])):
            for j in range(0, len(X[q])):
                nr+=1
                print("R",i,j,k,q," ", "X",k,i)
                ny+=1
                print("R",i,j,k,q," ", "X",q,j)
                nz+=1
print("#R=",nr)
print("#Y=",ny)
print("#Z=",nz)
print("#Additional Vars=",nr+ny+nz)

In [ ]:
numPosVars = m2q.bvc.get_NbPositioningVars(intmode=True)
numGapVars = m2q.bvc.get_NbGapVars(intmode=True)

print(numPosVars)
print(numGapVars)

In [ ]:
e2m = np.zeros((numPosVars+numGapVars+nr+ny+nz,numPosVars+numGapVars+nr+ny+nz))
print(e2m.shape)

In [ ]:
solution_vector = np.zeros((1, numPosVars+numGapVars+nr+ny+nz))

In [ ]:
print(m2q.bvc.W())

In [ ]:
# Move to the beggining of the matrix where the positions for R,Y and Z are
rMatPos = numPosVars+numGapVars

# Place the R*W variables on the R part of the matrix
posK = 0
posQ = 0
posI = 0
posJ = 0
for k in range(0,len(X)-1):
    posK += len(X[k])
    for q in range(k+1, len(X)):
        posQ += len(X[q])
        for i in range(0, len(X[k])):
            posI = 0
            posI = posK+i
            for j in range(0, len(X[q])):
                posJ = 0
                posJ = posQ+j
                matPos = rMatPos + posK + posQ + posI + posJ
                e2m[matPos][matPos] = m2q.bvc.w(i,j,k,q)


In [ ]:
# Place the X*Y, X*Z variables in the matrix multiplied by l_2 and W_ijkq

In [ ]:
yMatPos = (numPosVars+numGapVars)*2
zMatPos = (numPosVars+numGapVars)*3

posK = 0
posQ = 0
posI = 0
posJ = 0
for k in range(0,len(X)-1):
    for q in range(k+1, len(X)):
        for i in range(0, len(X[k])):
            # This loop would need to be extended for the binary version
            posI = 0
            posI = posK+i
            for j in range(0, len(X[q])):
                # This loop would need to be extended for the binary version
                posJ = 0
                posJ = posQ+j
                YmatPos = yMatPos + posK + posQ + posI + posJ
                ZmatPos = zMatPos + posK + posQ + posI + posJ
                e2m[posK+i][YmatPos] = -1. * m2q.bvc.l2() * m2q.bvc.w(i,j,k,q)
                e2m[posQ+j][YmatPos] = 2. * m2q.bvc.l2() * m2q.bvc.w(i,j,k,q)
                e2m[posQ+j][ZmatPos] = -1. * m2q.bvc.l2() * m2q.bvc.w(i,j,k,q)
        posQ += len(X[q])
    posK += len(X[k])


In [ ]:
# Place the bounds on Y and Z based on the McCormick envelopes

#YunderMin = 0
#YunderMax = X_ki + Rijkq*Lmax - Lmax

#YoverMin = X_ki
#YoverMax = Rijkq*Lmax

# This goes on the integer problem constraints matrix/array

In [ ]:
sum(e2m)

In [ ]:
m = m2q.bvc.m()
delta = 1
bVarRPos = m2q.bvc.get_NbPositioningVars() + m2q.bvc.get_NbGapVars()
bVarYPos = bRMatPos + nr # No need to transform R to binary as it only either (0,1)
# Since R is either 0,1 in the following loops need to take that into account...
bVarZPos = bYMatPos + ny*m2q.bvc.m()
e2bm = np.zeros((bVarZPos+nz*m, bVarZPos+nz*m))
posK = 0
posQ = 0
posI = 0
posJ = 0
for k in range(0,len(X)-1):
    for q in range(k+1, len(X)):
        for i in range(0, len(X[k])):
            posI = 0
            posI = posK+i
            for bi in range(0,m):
                for j in range(0, len(X[q])):
                    posJ = 0
                    posJ = posQ+j
                    for bj in range(0, m):
                        bRMatPos = bVarRPos
                        bYMatPos = bVarYPos
                        bZMatPos = bVarZPos
                        idx = posI+bi + posJ+bj +posK+posQ
                        
                        # -Yijkq*Xki
                        e2bm[bYMatPos+idx][posK+posI] += -1. * 2**(bi+bj)
                        # 3d * Yijkq
                        e2bm[bYMatPos+idx][bYMatPos+idx] += 3.*delta * 2**(bi+bj)
                        # d * Rijqk*Xki
                        e2bm[bRMatPos+idx][posK+posI] += 1.*delta * 2**(bi+bj)
                        # -2d * Yijkq*Rijkq
                        e2bm[bYMatPos+idx][bRMatPos+idx] += -2.*delta * 2**(bi+bj)
                        # 2d * Yijkq*Xki
                        e2bm[bYMatPos+idx][posK+posI+bi] += 2.*delta * 2**(bi+bj)
                        # -4d * Yijkq*Xqj
                        e2bm[bYMatPos+idx][posQ+posJ+bj] += -4.*delta * 2**(bi+bj)
                        # Zijkq*Xqj
                        e2bm[bZMatPos+idx][posQ+posJ+bj] += 1. * 2**(bi+bj)
                        # -3d * Zijkq
                        e2bm[bZMatPos+idx][bZMatPos+idx] += -3.*delta * 2**(bi+bj)
                        # -d * Rijkq*Xqj
                        e2bm[bRMatPos+idx][posQ+posJ+bj] += -1.*delta * 2**(bi+bj)
                        # 2d * Zijqk*Xqj
                        e2bm[bZMatPos+idx][posQ+posJ+bj] += 2.*delta * 2**(bi+bj)
                        # 2d * Zijkq*Rijkq
                        e2bm[bZMatPos+idx][bRMatPos+idx] += 2.*delta * 2**(bi+bj)

        posQ += len(X[q])
    posK += len(X[k])


In [ ]:
print(e2bm.nonzero())